# Construcción de red neuronal de clasificación binaria

In [17]:
import numpy as np
import pandas as pd
import h5py
from sklearn.datasets import load_breast_cancer

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

In [7]:
class red_neuronal_binaria():
  '''
    Parámetros de construcción:

    L (int): Número de capas de la red
    topologia (list int): Lista con el número de neuronas de cada capa
    N_epocas (int): Número de épocas para entrenar el algoritmo

    f_activacion (list str): Nombre de las funciones de activación a utilizar en cada capa

    N_train (int): Cantidad de datos de entrenamiento
    N_test (int): Cantidad de datos de prueba
    X_train (array): Matriz de características y datos de entrenamiento.
    y_train (array): Arreglo con las etiquetas de los datos de entrenamiento.
    X_test (array): Matriz de características y datos de prueba.
    y_test (array): Arreglo con las etiquetas de los datos de prueba.
    alpha (float): Tasa de aprendizaje para el gradiente descendente
  '''

  # Constructor de la clase
  # def __init__(self, L, topologia, N_epocas, f_activacion, N_train, N_test, X_train, y_train, X_test, y_test, alpha):
  def __init__(self, L, topologia, N_epocas, f_activacion, X_train, y_train, X_test, y_test, alpha):
    self.L = L
    self.topologia = topologia
    self.N_epocas = N_epocas
    self.f_activacion = f_activacion
    self.X = X_train
    self.y = y_train
    self.X_test = X_test
    self.y_test = y_test
    self.m = X_train.shape[1]
    self.m_test = X_test.shape[1]

    self.alpha = alpha

    # Definimos e inicializamos los pesos y los bias
    self.pesos = []
    self.bias = []
    self.init_pesos_bias()

  # Método para inicializar los pesos y sesgos de las neuronas en la red
  def init_pesos_bias(self):
    np.random.seed()
    pesos = []
    for i in range(self.L-1):
      # Los pesos obedecen una distribución normal estándar
      pesos.append(0.01*np.random.randn(self.topologia[i+1], self.topologia[i]))
    self.pesos = pesos

    bias = [np.zeros((self.topologia[i+1], 1)) for i in range(self.L-1)]
    self.bias = bias

  # Inicialización de las funciones de activación a partir de un X
  def func_activacion(self, f_name, X):
    if (f_name == 'sigmoid'):
      f = 1/(1+np.exp(-X))
      return f

    elif (f_name == 'relu'):
      f = np.maximum(0, X)
      return f

    elif (f_name == 'tanh'):
      f = np.tanh(X)
      return f

  # Derivada de la función de activación evaluada en x
  def derivada_f_activacion(self, f_name, X):
    if (f_name == 'sigmoid'):
      f = self.func_activacion('sigmoid', X)
      return f*(1-f)

    elif (f_name == 'relu'):
      X_copy = np.copy(X)
      X_copy[X_copy <= 0] = 0
      X_copy[X_copy > 0] = 1
      return X_copy

    elif (f_name == 'tanh'):
      f = self.func_activacion('tanh', X)
      return 1-f**2

  # Forward Pass: paso de la información (A y b) a través de todas las capas de la red
  # Z = pesos*A + b
  def forward_pass(self, wl, al_previo, bl, f_name):
    '''
      wl:         Arreglo de los pesos de la capa actual (l)
      al_previo:  Arreglo de la activación de la capa anterior (l-1)
      bl:         Arreglo de los sesgos de cada neurona de la capa actual (l)
      f_name:     Función de activación de la capa actual (l)

    Devuelve:
      al: Arreglo de la activación de la capa actual (l)
      zl: Arreglo de los valores de Z de la capa actual (l) -> Es el x de la función de activación
    '''
    zl = np.dot(wl, al_previo) + bl

    # Aplicamos la función de activación
    al = self.func_activacion(f_name, zl)
    return al, zl

  # Función de costo
  # al -> Activación de la última capa
  def costo(self, al):
    Li = self.y*np.log(al) + (1-self.y)*np.log(1-al)
    J = -1/self.m*np.sum(Li)
    return J

  # Backward Pass: Retropropagación del error a través de la red
  '''
    La función del Backward Pass es realizar propagar la información en sentido inverso (desde la salida)
    para calcular el gradiente de la función de pérdida con respecto a los pesos y sesgos.
  '''
  # Comenzamos el proceso de retropropagación desde la última capa de la red neuronal
  def backward_pass_end(self, aL, aL_previo, Y, wL, bL):
    '''
      aL: Activación de la última capa
      aL_previo: Activación de la capa anterior
      wL: Pesos de la última capa
      bL: Sesgos de la última capa
    Devuelve:
      wl: Los pesos actualizados de la última capa
      bl: Los sesgos actualizados de la última capa
    '''
    err_L = aL-Y      # Error de la capa final
    dJdw = 1/self.m*np.dot(err_L, aL_previo.T)

    # Respecto al vector sesgo, sumamos por filas y mantenemos dimensiones matriciales
    dJdb = 1/self.m*np.sum(err_L, axis=1, keepdims=True)

    # Gradiente descendente
    wL = wL - self.alpha*dJdw
    bL = bL - self.alpha*dJdb

    return wL, bL

  # Backward pass para capas ocultas
  def backward_pass(self, err_lnext, al_previo, wl, wl_next, zl, bl, f_name):
    '''
      al_previo:  Arreglo de la activación de la capa anterior (l-1)
      wl:         Arreglo de los pesos de la capa actual (l)
      wl_next:    Arreglo de los pesos de la capa siguiente (l+1)
      err_lnext:  Error de la capa siguiente
      zl:         Z de la capa actual para aplicar la función de activación f_name
      bl:         Arreglo de los sesgos de cada neurona de la capa actual (l)

    Devuelve:
      wl, bl, y err_l: error de la capa actual (l)
    '''
    err_l = np.dot(wl_next.T, err_lnext)*self.derivada_f_activacion(f_name, zl)
    dJdw = 1/self.m*np.dot(err_l, al_previo.T)
    dJdb = 1/self.m*np.sum(err_l, axis=1, keepdims=True)  # Sumamos por filas y mantenemos dimensiones matriciales

    # Gradiente descendente
    wl = wl - self.alpha*dJdw
    bl = bl - self.alpha*dJdb

    return wl, bl, err_l

  # Entrenamiento de la red neuronal a partir de la propagación hacia adelante y hacia atrás
  def train(self):
    # Por cada época
    for epoca in range(self.N_epocas):
      a = [self.X]
      z = []

      # Propagación hacia adelante
      for i in range(self.L-1):
        al, zl = self.forward_pass(self.pesos[i], a[-1], self.bias[i], self.f_activacion[i])
        a.append(al)
        z.append(zl)

      # Costo
      J = self.costo(a[-1])

      # Propagación hacia atrás
      err_l = a[-1] - self.y
      for i in reversed(range(self.L-1)):     # Iteración inversa
        if (i == self.L-2):
          self.pesos[i], self.bias[i] = self.backward_pass_end(a[-1], a[-2], self.y, self.pesos[i], self.bias[i])
        else:
          self.pesos[i], self.bias[i], err_l = self.backward_pass(err_l, a[i], self.pesos[i], self.pesos[i+1], z[i], self.bias[i], self.f_activacion[i])

  # Una vez entrenada la red, realizamos las predicciones
  def predict(self, X):
    a = X     # Se activa la red con los datos de entrada
    for i in range(self.L-1):
      a, z = self.forward_pass(self.pesos[i], a, self.bias[i], self.f_activacion[i])

    # Retornamos la activación de la última capa (resultado predicho)
    return a

  # Exactitud del modelo de acuerdo a los datos proporcionados
  def score(self, X, Y):
    pred = self.predict(X)    # Probabilidades

    # Pasamos de probabilidades a clases binarias
    pred = (pred > 0.5).astype(int)

    return np.mean(pred == Y)

## Importamos los datasets

In [8]:
import gdown

url = 'https://drive.google.com/drive/folders/1cgFsOb4kQPobYqAJJEOJihp0JJKHFXIn?usp=sharing'

# Dataset de gatitos
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/dataset_gatos_lab7/test_catvnoncat.h5',
 '/content/dataset_gatos_lab7/train_catvnoncat.h5']

### Dataset de gatos

In [9]:
data_train = "./dataset_gatos_lab7/train_catvnoncat.h5"
train_dataset = h5py.File(data_train, "r")

data_test = "./dataset_gatos_lab7/test_catvnoncat.h5"
test_dataset = h5py.File(data_test, "r")

xtrain_classes, xtrain, train_label =\
train_dataset["list_classes"], train_dataset["train_set_x"], train_dataset["train_set_y"]

test_classes, xtest, test_label =\
test_dataset["list_classes"], test_dataset["test_set_x"], test_dataset["test_set_y"]

In [10]:
''' Ejecución en el dataset de fotos de gatos '''

y_train = np.array(train_label)
X_train = (np.reshape(xtrain, (xtrain.shape[0], -1))/255).T

y_test = np.array(test_label)
X_test = (np.reshape(xtest, (xtest.shape[0], -1))/255).T

# Parámetros de la red
L = 4  # Número de capas
topologia = [X_train.shape[0], 10, 10, 1]   # Neuronas de la 1er capa=# de características, 2 capas ocultas de 10 neuronas, la última de 1
fs_activacion = ['relu', 'sigmoid', 'sigmoid', 'sigmoid']
N_epocas = 1000
N_train = X_train.shape[1]
N_test = X_test.shape[1]
alpha = 0.01              # Tasa de aprendizaje

# Crear y entrenar el modelo
model = red_neuronal_binaria(L, topologia, N_epocas, fs_activacion, X_train, y_train, X_test, y_test, alpha)
model.train()

# Evaluar el modelo
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.6555023923444976
Test Score: 0.34


El puntaje de entrenamiento es 0.65, pero el puntaje de prueba es 0.34. Esto evidencia un sobreajuste en el modelo.

### Dataset de cáncer de mama

In [18]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [25]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(df, df["target"]):
  strat_train_set = df.loc[train_index]
  strat_test_set = df.loc[test_index]

df_train = strat_train_set
df_test = strat_test_set

# División de los datos
X_train = df_train.drop('target', axis=1).to_numpy()
y_train = df_train['target'].to_numpy()

X_test = df_test.drop('target', axis=1).to_numpy()
y_test = df_test['target'].to_numpy()

# Estandarización de los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.T
y_train = y_train.reshape(-1, 1)
y_train = y_train.T

X_test = X_test.T
y_test = y_test.reshape(-1, 1)
y_test = y_test.T

In [27]:
# Parámetros de la red
L = 4
topologia = [X_train.shape[0], 10, 5, 3, 1]
fs_activacion = ['relu', 'sigmoid', 'sigmoid', 'sigmoid']
N_epocas = 1000
N_train = X_train.shape[1]
N_test = X_test.shape[1]
alpha = 0.01

# Creación y entrenamiento del modelo
model = red_neuronal_binaria(L, topologia, N_epocas, fs_activacion, X_train, y_train, X_test, y_test, alpha)
model.train()

# Evaluación del modelo
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.6263736263736264
Test Score: 0.631578947368421


Ambos resultados muestran un rendimiento bastante bajo para el modelo, pues la métrica de exactitud no se acerca a 1, por lo que no logra clasificar muchos falsos positivos y negativos.